# Visual Covariance Test

$$
% latex macros.  this does not work when converting to latex.
\newcommand{\bm}[1]{\boldsymbol{#1}}
\DeclareMathOperator{\cov}{cov}
\DeclareMathOperator{\diag}{diag}
\newcommand{\ud}{\mathrm{d}}
\newcommand{\XX}{{\bm X}}
\newcommand{\VV}{{\bm V}}
\newcommand{\N}{\mathcal{N}}
\newcommand{\mmu}{{\bm \mu}}
\newcommand{\SSi}{{\bm \Sigma}}
\newcommand{\ssi}{{\bm \sigma}}
$$

Pick a covariance type. Then for $x_t$ and $v_t = \ud x_t/\ud t$, visually check that
$$
\begin{aligned}
\cov(x_t, v_s) & = \int_0^t \cov(v_u, v_s) \, \ud u \\
\cov(x_t, x_s) & = \int_0^t\int_0^s \cov(v_u, v_w) \, \ud u\, \ud w
\end{aligned}
$$
by comparing analytic formula to numerical integration with `scipy.integrate.quad` and `scipy.integrate.dblquad`.

In [1]:
import os
os.chdir('C:\\Users\\mohan\\Documents\\probDE')
#os.chdir('../')

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import uniform as runif
from scipy import integrate
import BayesODE as bo
from Tests.ode_bayes_star import ode_bayes_star

In [ ]:
npts = 50
# pick covariance type
type = "ex"
if type == "se":
    cov_vv = bo.cov_vv_se
    cov_xv = bo.cov_xv_se
    cov_xx = bo.cov_xx_se
elif type == "ex":
    cov_vv = bo.cov_vv_ex
    cov_xv = bo.cov_xv_ex
    cov_xx = bo.cov_xx_ex
elif type == "re":
    cov_vv = bo.cov_vv_re
    cov_xv = bo.cov_xv_re
    cov_xx = bo.cov_xx_re
else:
    raise ValueError("Invalid covariance type.")
# plot cov_vv
tmax = runif(low = 0, high = 10, size = 1)
gamma = runif(0, tmax/10, 1)
tseq = np.linspace(start = 0, stop = tmax, num = npts)
av = np.zeros((npts, 1))
for ii in range(0,npts):
    av[ii] = cov_vv(0, tseq[ii], gamma)
plt.rcParams['figure.figsize'] = [10, 6]
plt.subplot(3,2,(1,2))
plt.plot(tseq,av)
# integrate once and compare to analytic solution
t = runif(0, tmax, 1) # first time point
s = runif(0, tmax, 1) # second time point
tseq = np.linspace(start = 0, stop = tmax, num = npts)
xv_an = np.zeros((npts,1)) # analytic solution
xv_nu = xv_an # numeric solution
for ii in range(0,npts):
    xv_an[ii] = cov_xv(tseq[ii], s, gamma)
    xv_nu[ii],err = integrate.quad(cov_vv, 0, tseq[ii], args=(s,gamma))
plt.subplot(3,2,3)
plt.plot(tseq,xv_an,'r',tseq,xv_nu,'b')
plt.subplot(3,2,4)
plt.plot(tseq,abs(xv_an-xv_nu))
# integrate twice and compare to analytic solution
xx_an = np.zeros((npts,1)) # analytic solution
xx_nu = xx_an # numeric solution
for ii in range(0,npts):
    xx_an[ii] = cov_xx(tseq[ii], s, gamma)
    xx_nu[ii],err = integrate.dblquad(cov_vv, 0, tseq[ii], 0, s, args = (gamma))
plt.subplot(3,2,5)
plt.plot(tseq,xx_an,'r',tseq,xx_nu,'b')
plt.subplot(3,2,6)
plt.plot(tseq,abs(xx_an-xx_nu))

# Updating Algorithm Test

Let $T = (t_1, \ldots, t_N)$ denote the evaluation time points, and $\XX$ and $\VV$ denote vectors of length $N$ corresponding to $x(T)$ and $v(T)$.  The solution prior is then expressed as
$$
\begin{bmatrix} \XX \\ \VV \end{bmatrix} \sim \N\left(\begin{bmatrix} \mmu_X \\ \mmu_V \end{bmatrix}, \begin{bmatrix} \SSi_{XX} & \SSi_{XV} \\ \SSi_{VX} & \SSi_{VV} \end{bmatrix} \right).
$$
Now suppose that the $N$ model interrogations $\VV_\star$ and their predictive variances $\ssi_\star^2$ are given in advance.  Then joint distribution of $(\XX, \VV, \VV_\star)$ is
$$
\begin{bmatrix} \XX \\ \VV \\ \VV_\star \end{bmatrix} \sim \N\left(\begin{bmatrix} \mmu_X \\ \mmu_V \\ \mmu_V \end{bmatrix}, \begin{bmatrix} \SSi_{XX} & \SSi_{XV} & \SSi_{XV} \\ \SSi_{VX} & \SSi_{VV} & \SSi_{VV} \\ \SSi_{VX} & \SSi_{VV} & \SSi_{VV} + \diag(\ssi_\star^2) \end{bmatrix} \right),
$$
from which we can deduce calculate the mean and variance of the final update (normal) distribution, $p(\XX \mid \VV_\star)$.  This suggests the following unit test for the ODE solver:

1. Modify `ode_bayes` such that it accepts an optional input of $\VV_\star$ and $\ssi_\star^2$ (please use argument names consistent with the rest of the function).
2. Write a function to compute the mean and variance of $p(\XX \mid \VV_\star)$ based on $(\mmu_X, \mmu_V, \SSi_{XX}, \SSi_{XV}, \SSi_{VV}, \VV_\star)$.
3. Run `ode_bayes` with predetermined $(\VV_\star, \ssi_\star^2)$, and check that the mean and variance output is identical to that of step 2.

In [4]:
#Toy example
def f(x,t):
    return  3*(t+1/4) - x/(t+1/4)

a = 0
b = 4.75
x0 = 0
N = 100
tseq = np.linspace(a, b, N)
w = gamma = 1.0642211055276383
alpha = N/10

mu_v = np.zeros((N)) # prior mean of v(tseq)
mu_x = x0 + mu_v # prior mean of x(tseq)
Sigma_vv, Sigma_xx, Sigma_xv = bo.cov_prior(tseq, 'exp', gamma, alpha)

In [ ]:
x = integrate.odeint(f,x0,tseq)
x = np.ndarray.flatten(x)
v = np.array([f(xl,tseq[i]) for i, xl in enumerate(x)])
sigma_star_avg = (max(v) - min(v))*0.05

upper = 3*sigma_star_avg/2
lower = sigma_star_avg/2

sigma_star = runif(lower, upper)
vstar = v+sigma_star*np.random.normal(0,1)

In [ ]:
mu_star, Sigma_star = ode_bayes_star(mu_x, mu_v, Sigma_vv, Sigma_xx, Sigma_xv, vstar)

In [ ]:
_, mu_x, Sigma_xx = bo.ode_bayes(f, tseq, x0, Sigma_vv, Sigma_xx, Sigma_xv, vstar)

In [ ]:
np.allclose(mu_star, mu_x), np.allclose(Sigma_star, Sigma_xx)

## Implementation via the Kalman Filter

For the special case of an exponential kernel (or equivalently, exponential autocorrelation of the solution derivative), the Bayesian ODE solver can be efficiently implemented via Kalman filtering and smoothing algorithms.  To see this, the first step is to derive the conditional distribution $p(y_t \mid y_s)$, where $y_t = (x_t, v_t)$ and $0 < s < t$.

In [6]:
import scipy.linalg as scl

def mvCond(mu, Sigma, icond):
    """
    For y ~ N(mu, Sigma), returns A, b, and V such that y[~icond] | y[icond] ~ N(A * y[icond] + b, V).
    """
    # if y1 = y[~icond] and y2 = y[icond], should have A = Sigma12 * Sigma22^{-1}
    A = np.dot(Sigma[np.ix_(~icond, icond)],scl.cho_solve(scl.cho_factor(Sigma[np.ix_(icond,icond)]), np.identity(sum(icond))))
    b = mu[~icond] - np.dot(A, mu[icond]) # mu1 - A * mu2
    V = Sigma[np.ix_(~icond,~icond)] - np.dot(A, Sigma[np.ix_(icond,~icond)]) # Sigma11 - A * Sigma21
    return A, b, V

# simple test
mu = np.array([1, 2, 3, 4, 5])
Sigma = np.diag([1, 2, 3, 4, 5])
icond = np.array([True, True, True, False, False])
i,o,p = mvCond(mu, Sigma, icond)

In [11]:
def mvCondExp(t1, t2, x0, gamma):
    """Calculates A, b, V for p(y(t2) | y(t1)), where 0 < t1 < t2."""
    mu = np.array([x0, 0, x0, 0])
    Sigma = np.zeros((4,4))
    # cov(x1,:)
    Sigma[0,0] = bo.cov_xx_ex(t1, t1, gamma) # x1, x1
    Sigma[0,1] = bo.cov_xv_ex(t1, t1, gamma) # x1, v1
    Sigma[0,2] = bo.cov_xx_ex(t1, t2, gamma) # x1, x2
    Sigma[0,3] = bo.cov_xv_ex(t1, t2, gamma) # x1, v2
    # cov(v1,:)
    Sigma[1,0] = Sigma[0,1]                  # v1, x1
    Sigma[1,1] = bo.cov_vv_ex(t1, t1, gamma) # v1, v1
    Sigma[1,2] = bo.cov_xv_ex(t2, t1, gamma) # v1, x2 <- this is the calculation I'm suspicious of, i.e., time_x > time_v
    Sigma[1,3] = bo.cov_vv_ex(t1, t2, gamma) # v1, v2
    # cov(x2,:)
    Sigma[2,0] = Sigma[0,2]                  # x2, x1
    Sigma[2,1] = Sigma[1,2]                  # x2, v1
    Sigma[2,2] = bo.cov_xx_ex(t2, t2, gamma) # x2, x2
    Sigma[2,3] = bo.cov_xv_ex(t2, t2, gamma) # x2, v2
    # cov(v2,:)
    Sigma[3,0] = Sigma[0,3]                  # v2, x1
    Sigma[3,1] = Sigma[1,3]                  # v2, v1
    Sigma[3,2] = Sigma[2,3]                  # v2, x2
    Sigma[3,3] = bo.cov_vv_ex(t2, t2, gamma) # v2, v2
    return Sigma # stop here for now...

# should give the same answer for any x0 and fixed dt = t2 - t1...
dt = 1
t1 = 1
t2 = t1 + dt
x0 = 2
scl.eigvalsh(mvCondExp(t1, t2, x0, 1)) # should always be positive...

array([ 0.05919278,  0.24789939,  0.76248666,  3.93685063])

In [12]:
def mvCondRE(t1, t2, x0, gamma):
    """Calculates A, b, V for p(y(t2) | y(t1)), where 0 < t1 < t2."""
    mu = np.array([x0, 0, x0, 0])
    Sigma = np.zeros((4,4))
    # cov(x1,:)
    Sigma[0,0] = bo.cov_xx_re(t1, t1, gamma) # x1, x1
    Sigma[0,1] = bo.cov_xv_re(t1, t1, gamma) # x1, v1
    Sigma[0,2] = bo.cov_xx_re(t1, t2, gamma) # x1, x2
    Sigma[0,3] = bo.cov_xv_re(t1, t2, gamma) # x1, v2
    # cov(v1,:)
    Sigma[1,0] = Sigma[0,1]                  # v1, x1
    Sigma[1,1] = bo.cov_vv_re(t1, t1, gamma) # v1, v1
    Sigma[1,2] = bo.cov_xv_re(t2, t1, gamma) # v1, x2 <- this is the calculation I'm suspicious of, i.e., time_x > time_v
    Sigma[1,3] = bo.cov_vv_re(t1, t2, gamma) # v1, v2
    # cov(x2,:)
    Sigma[2,0] = Sigma[0,2]                  # x2, x1
    Sigma[2,1] = Sigma[1,2]                  # x2, v1
    Sigma[2,2] = bo.cov_xx_re(t2, t2, gamma) # x2, x2
    Sigma[2,3] = bo.cov_xv_re(t2, t2, gamma) # x2, v2
    # cov(v2,:)
    Sigma[3,0] = Sigma[0,3]                  # v2, x1
    Sigma[3,1] = Sigma[1,3]                  # v2, v1
    Sigma[3,2] = Sigma[2,3]                  # v2, x2
    Sigma[3,3] = bo.cov_vv_re(t2, t2, gamma) # v2, v2
    return Sigma # stop here for now...

# should give the same answer for any x0 and fixed dt = t2 - t1...
dt = 1
t1 = 1
t2 = t1 + dt
x0 = 2
scl.eigvalsh(mvCondRE(t1, t2, x0, 1)) # should always be positive...

array([ 0.06451011,  0.28664054,  1.47350766,  9.1753417 ])

In [13]:
def mvCondSE(t1, t2, x0, gamma):
    """Calculates A, b, V for p(y(t2) | y(t1)), where 0 < t1 < t2."""
    mu = np.array([x0, 0, x0, 0])
    Sigma = np.zeros((4,4))
    # cov(x1,:)
    Sigma[0,0] = bo.cov_xx_se(t1, t1, gamma) # x1, x1
    Sigma[0,1] = bo.cov_xv_se(t1, t1, gamma) # x1, v1
    Sigma[0,2] = bo.cov_xx_se(t1, t2, gamma) # x1, x2
    Sigma[0,3] = bo.cov_xv_se(t1, t2, gamma) # x1, v2
    # cov(v1,:)
    Sigma[1,0] = Sigma[0,1]                  # v1, x1
    Sigma[1,1] = bo.cov_vv_se(t1, t1, gamma) # v1, v1
    Sigma[1,2] = bo.cov_xv_se(t2, t1, gamma) # v1, x2 <- this is the calculation I'm suspicious of, i.e., time_x > time_v
    Sigma[1,3] = bo.cov_vv_se(t1, t2, gamma) # v1, v2
    # cov(x2,:)
    Sigma[2,0] = Sigma[0,2]                  # x2, x1
    Sigma[2,1] = Sigma[1,2]                  # x2, v1
    Sigma[2,2] = bo.cov_xx_se(t2, t2, gamma) # x2, x2
    Sigma[2,3] = bo.cov_xv_se(t2, t2, gamma) # x2, v2
    # cov(v2,:)
    Sigma[3,0] = Sigma[0,3]                  # v2, x1
    Sigma[3,1] = Sigma[1,3]                  # v2, v1
    Sigma[3,2] = Sigma[2,3]                  # v2, x2
    Sigma[3,3] = bo.cov_vv_se(t2, t2, gamma) # v2, v2
    return Sigma # stop here for now...

# should give the same answer for any x0 and fixed dt = t2 - t1...
dt = 1
t1 = 1
t2 = t1 + dt
x0 = 2
scl.eigvalsh(mvCondSE(t1, t2, x0, 1)) # should always be positive...

array([ -1.44485376,   0.21247491,   1.01510326,  11.57239126])

In [15]:
mvCondExp(t1, t2, x0, 1)

array([[ 0.73575888,  0.63212056,  1.13533528,  0.23254416],
       [ 0.63212056,  1.        ,  1.26424112,  0.36787944],
       [ 1.13533528,  1.26424112,  2.27067057,  0.86466472],
       [ 0.23254416,  0.36787944,  0.86466472,  1.        ]])

In [16]:
mvCondRE(t1, t2, x0, 1)

array([[ 1.66666667,  1.5       ,  2.66666667,  0.5       ],
       [ 1.5       ,  2.        ,  3.        ,  1.        ],
       [ 2.66666667,  3.        ,  5.33333333,  2.        ],
       [ 0.5       ,  1.        ,  2.        ,  2.        ]])

In [14]:
mvCondSE(t1, t2, x0, 1)

array([[ 1.70213557,  1.63519859,  4.6892348 ,  1.01222403],
       [ 1.63519859,  1.77245385,  3.27039718,  1.38038845],
       [ 4.6892348 ,  3.27039718,  6.10807241,  2.64742262],
       [ 1.01222403,  1.38038845,  2.64742262,  1.77245385]])

In [ ]:
from pykalman import KalmanFilter

In [ ]:
tseq2 = np.linspace(a,b,50)
def mvCond(tseq, x0):
    N = len(tseq)
    mu = [x0, 0]*N
    Sigma = np.matrix([[0]*N*2]*N*2)
    for i in range(N):
        for j in range(N):
            Sigma[2*i, 2*j] = bo.cov_xx_ex(tseq[i], tseq[j], gamma)
            Sigma[2*i, 2*j+1] = bo.cov_xv_ex(tseq[i], tseq[j], gamma)
            Sigma[2*i+1, 2*j] = bo.cov_xv_ex(tseq[j], tseq[i], gamma)
            Sigma[2*i+1, 2*j+1] = bo.cov_vv_ex(tseq[i], tseq[j], gamma)
    Sigma_00 = Sigma[0:2*N-2, 0:2*N-2]
    Sigma_10 = Sigma[2*N-2:, 0:2*N-2]
    Sigma_01 = Sigma[0:2*N-2, 2*N-2:]
    Sigma_11 = Sigma[2*N-2:, 2*N-2:]
    A = Sigma_10*np.linalg.pinv(Sigma_00)
    b = mu[2*N-2:]
    V = Sigma_11 - Sigma_10*np.linalg.pinv(Sigma_00)*Sigma_01
    return A,b,V
A, b, V = mvCond(tseq2, x0)
A

In [ ]:
Sigma_vv_00 = bo.cov_vv_ex(tseq[0], tseq[0], gamma)
Sigma_xx_00 = bo.cov_xx_ex(tseq[0], tseq[0], gamma)
Sigma_xv_00 = bo.cov_xv_ex(tseq[0], tseq[0], gamma)
Sigma_vv_10 = bo.cov_vv_ex(tseq[1], tseq[0], gamma)
Sigma_xx_10 = bo.cov_xx_ex(tseq[1], tseq[0], gamma)
Sigma_xv_10 = bo.cov_xv_ex(tseq[1], tseq[0], gamma)

Sigma_00 = np.matrix([[Sigma_xx_00, Sigma_xv_00],[Sigma_xv_00, Sigma_vv_00]])
Sigma_10 = np.matrix([[Sigma_xx_10, Sigma_xv_10],[Sigma_xv_10, Sigma_vv_10]])
A = Sigma_10*np.linalg.pinv(Sigma_00)

Sigma_vv_11 = bo.cov_vv_ex(tseq[1], tseq[1], gamma)
Sigma_xx_11 = bo.cov_xx_ex(tseq[1], tseq[1], gamma)
Sigma_xv_11 = bo.cov_xv_ex(tseq[1], tseq[1], gamma)
Sigma_vv_01 = bo.cov_vv_ex(tseq[0], tseq[1], gamma)
Sigma_xx_01 = bo.cov_xx_ex(tseq[0], tseq[1], gamma)
Sigma_xv_01 = bo.cov_xv_ex(tseq[0], tseq[1], gamma)

Sigma_11 = np.matrix([[Sigma_xx_11, Sigma_xv_11],[Sigma_xv_11, Sigma_vv_11]])
Sigma_01 = np.matrix([[Sigma_xx_01, Sigma_xv_01],[Sigma_xv_01, Sigma_vv_01]])
CC = Sigma_11 - Sigma_10*np.linalg.pinv(Sigma_00)*Sigma_01

In [ ]:
A

In [ ]:
CC

In [ ]:
def bayes_ode_kalman(fun, x0, tseq, gamma, alpha, A, CC):
    y0 = [x0, f(x0, tseq[0])]
    mu = [y0]*N
    Sigma = [[[0, 0], [0,0]]]*N
    yt = np.array([y0]*N)
    mu_nn = np.array((A*np.matrix(mu[0]).T)).T
    Sigma_nn = A*Sigma[0]*A.T + CC
    kf = KalmanFilter(initial_state_mean=mu[0], initial_state_covariance=Sigma[0], n_dim_obs=2) #Initialization of Kalman Filter
    for n in range(N-1):
        mu_nn = np.array((A*np.matrix(mu[n]).T)).T
        Sigma_nn = A*Sigma[n]*A.T + CC
        
        #Model Interrogation Step
        xt = np.random.normal(mu_nn[0][0], Sigma_nn[0,0])
        vt = fun(xt, tseq[n+1])
        yt[n+1] = [xt,vt]
        mu[n+1], Sigma[n+1] = kf.filter_update(mu_nn[0], Sigma_nn, yt[n+1])
    
    kf2 = KalmanFilter(initial_state_mean=mu[0], initial_state_covariance=Sigma[0], n_dim_obs=2)
    yt_mean = kf2.smooth(yt)[0]
    
    return mu, yt_mean

In [ ]:
u,v = bayes_ode_kalman(f, x0, tseq, gamma, alpha, A, CC)
